In [11]:
import pandas as pd
import json
import os

### Recompute pass@1 scores

In [110]:
model = "allenai--OLMo-2-1124-7B-Instruct" #  "Qwen--Qwen2.5-7B-Instruct-1M" #

In [111]:
def is_correct(x, subset="base"):
    if subset == "base":
        return True if x['base_status'] == "pass" else False
    elif subset == "plus":
        return True if x['base_status'] == "pass" and x['plus_status'] == "pass" else False
    else:
        raise NotImplementedError

In [112]:
baseline_file = f"results/a0/mbpp/{model}_vllm_temp_0.0.eval_results.json"
baseline = json.load(open(baseline_file))
baseline_results = [result[0] for result in baseline['eval'].values()]
baseline_df = pd.DataFrame(baseline_results).sort_values(by='task_id')
print(len(baseline_df))
baseline_df['baseline_base_correct'] = baseline_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
baseline_df['baseline_plus_correct'] = baseline_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)
# print(baseline_df.baseline_base_correct.value_counts())
# print(baseline_df.baseline_plus_correct.value_counts())
baseline_df.head()

378


,task_id,solution,base_status,plus_status,base_fail_tests,plus_fail_tests,baseline_base_correct,baseline_plus_correct
53,Mbpp/100,def next_smallest_palindrome(n):\n # Functi...,pass,pass,[],[],True,True
55,Mbpp/101,"def kth_element(arr, k):\n """"""\n Functio...",pass,pass,[],[],True,True
46,Mbpp/102,def snake_to_camel(snake_case_string):\n # ...,pass,fail,[],[[___python_program]],True,False
114,Mbpp/103,"def eulerian_num(n, m):\n """"""\n Calculat...",fail,timeout,"[[4, 1]]",[],False,False
44,Mbpp/104,def sort_sublists(sublists):\n # Sort each ...,pass,pass,[],[],True,True


In [113]:
# calculate baseline accuracy
baseline_base_correct = baseline_df.baseline_base_correct.value_counts()[True]
baseline_plus_correct = baseline_df.baseline_plus_correct.value_counts()[True]
baseline_base_accuracy = baseline_base_correct / len(baseline_df)
baseline_plus_accuracy = baseline_plus_correct / len(baseline_df)
print(f"Baseline accuracy: {baseline_base_accuracy}, {baseline_plus_accuracy}")

Baseline accuracy: 0.43915343915343913, 0.36772486772486773


In [114]:
# get all folders in results/a1
folders = [f for f in os.listdir("results/a1") if os.path.isdir(os.path.join("results/a1", f))]
print(folders)
print(len(folders))
output_file = f"results/metrics/{model}_scores.csv"

['mbpp_just_feedback_gpt-3.5-turbo', 'mbpp_just_retrieval_as_feedback', 'mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_with_feedback_gpt-3.5-turbo', 'mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question']
8


In [115]:
all_rows = []
for setup in folders:

    results_file = f"results/a1/{setup}/{model}_vllm_temp_0.0.eval_results.json"
    model_results = json.load(open(results_file))
    results = [result[0] for result in model_results['eval'].values()]
    exp_df = pd.DataFrame(results)

    exp_df['baseline_base_correct'] = exp_df.apply(lambda x: baseline_df[baseline_df.task_id == x.task_id].baseline_base_correct.values[0], axis=1)
    exp_df['baseline_plus_correct'] = exp_df.apply(lambda x: baseline_df[baseline_df.task_id == x.task_id].baseline_plus_correct.values[0], axis=1)

    exp_df['base_correct'] = exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    exp_df['plus_correct'] = exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)
    # print(exp_df.head())

    # get the additional correct examples in exp_df that are wrong in baseline
    exp_add_base_correct = len(exp_df[exp_df.base_correct & ~exp_df.baseline_base_correct])
    exp_add_plus_correct = len(exp_df[exp_df.plus_correct & ~exp_df.baseline_plus_correct])
    print("Wrong to correct:", exp_add_base_correct, exp_add_plus_correct)

    # check correct --> wrong
    # base_correct_to_wrong = len(exp_df[~exp_df.base_correct &~exp_df.baseline_base_correct])
    # plus_correct_to_wrong = len(exp_df[~exp_df.plus_correct &~exp_df.baseline_plus_correct])
    # print("Correct to wrong:", base_correct_to_wrong, plus_correct_to_wrong)

    exp_base_accuracy = (baseline_base_correct + exp_add_base_correct) / len(exp_df)
    exp_plus_accuracy = (baseline_plus_correct + exp_add_plus_correct) / len(exp_df)
    print(f"{setup} acc: {exp_base_accuracy}, {exp_plus_accuracy}")

    row = {'setup': setup, 'model': model, 'base_accuracy': exp_base_accuracy, 'plus_accuracy': exp_plus_accuracy}
    # print(row)
    all_rows.append(row)
baseline_row = {'setup': 'baseline', 'model': model, 'base_accuracy': baseline_base_accuracy, 'plus_accuracy': baseline_plus_accuracy}
all_rows.append(baseline_row)


Wrong to correct: 80 60
mbpp_just_feedback_gpt-3.5-turbo acc: 0.6507936507936508, 0.5264550264550265
Wrong to correct: 16 14
mbpp_just_retrieval_as_feedback acc: 0.48148148148148145, 0.40476190476190477
Wrong to correct: 12 13
mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.4708994708994709, 0.4021164021164021
Wrong to correct: 1 1
mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question acc: 0.4417989417989418, 0.37037037037037035
Wrong to correct: 4 6
mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.4497354497354497, 0.3835978835978836
Wrong to correct: 19 16
mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.4894179894179894, 0.41005291005291006
Wrong to correct: 70 54
mbpp_with_feedback_gpt-3.5-turbo acc: 0.6243386243386243, 0.5105820105820106
Wrong to correct: 21 20
mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.4947089947089947, 0.42063492063492064


In [116]:
# save all rows to a csv file
df = pd.DataFrame(all_rows)
df.to_csv(output_file, index=False)
df.head()

,setup,model,base_accuracy,plus_accuracy
0,mbpp_just_feedback_gpt-3.5-turbo,allenai--OLMo-2-1124-7B-Instruct,0.650794,0.526455
1,mbpp_just_retrieval_as_feedback,allenai--OLMo-2-1124-7B-Instruct,0.481481,0.404762
2,mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_ch...,allenai--OLMo-2-1124-7B-Instruct,0.470899,0.402116
3,mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_n...,allenai--OLMo-2-1124-7B-Instruct,0.441799,0.370370
4,mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_q...,allenai--OLMo-2-1124-7B-Instruct,0.449735,0.383598


### Compute helpfulness, leakage frequency and non-leaking helpfulness

In [98]:
output_file = f"results/metrics/{model}_eval_metrics.csv"
print(output_file)

results/metrics/allenai--OLMo-2-1124-7B-Instruct_eval_metrics.csv


In [99]:
exp_ids = [
    "Teacher Feedback",
    "Teacher Feedback (No Cheat No Question)",
    "Teacher Feedback with RAG",
    "RAG Feedback",
]

leak_exps = [
    'mbpp_just_feedback_gpt-3.5-turbo',
    'mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question', 
    'mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question',
    'mbpp_just_retrieval_as_feedback', 
    
]

actual_exps = [
    'mbpp_with_feedback_gpt-3.5-turbo', 
    'mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question', 
    'mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question',
    'mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question', 
]


In [100]:
all_rows = []
for exp_id, leak_exp, actual_exp in zip(exp_ids, leak_exps, actual_exps):
    print(f"{exp_id} (actual = {actual_exp}, leak exp = {leak_exp}):")
    
    leak_exp_file = f"results/a1/{leak_exp}/{model}_vllm_temp_0.0.eval_results.json"
    actual_exp_file = f"results/a1/{actual_exp}/{model}_vllm_temp_0.0.eval_results.json"

    leak_exp_results = json.load(open(leak_exp_file))
    actual_exp_results = json.load(open(actual_exp_file))

    leak_exp_results = [result[0] for result in leak_exp_results['eval'].values()]
    actual_exp_results = [result[0] for result in actual_exp_results['eval'].values()]

    leak_exp_df = pd.DataFrame(leak_exp_results)
    actual_exp_df = pd.DataFrame(actual_exp_results)


    leak_exp_df['base_correct'] = leak_exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    leak_exp_df['plus_correct'] = leak_exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)


    actual_exp_df['baseline_base_correct'] = actual_exp_df.apply(lambda x: baseline_df[baseline_df.task_id == x.task_id].baseline_base_correct.values[0], axis=1)
    actual_exp_df['baseline_plus_correct'] = actual_exp_df.apply(lambda x: baseline_df[baseline_df.task_id == x.task_id].baseline_plus_correct.values[0], axis=1)

    actual_exp_df['base_correct'] = actual_exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    actual_exp_df['plus_correct'] = actual_exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)

    actual_exp_df['base_leaked'] = actual_exp_df.apply(lambda x: leak_exp_df[leak_exp_df.task_id == x.task_id].base_correct.values[0], axis=1)
    actual_exp_df['plus_leaked'] = actual_exp_df.apply(lambda x: leak_exp_df[leak_exp_df.task_id == x.task_id].plus_correct.values[0], axis=1)

    # helpfulness 
    actual_exp_df['base_helpful'] = actual_exp_df.apply(lambda x: x.base_correct & ~x.baseline_base_correct, axis=1)
    actual_exp_df['plus_helpful'] = actual_exp_df.apply(lambda x: x.plus_correct & ~x.baseline_plus_correct, axis=1)

    # print(actual_exp_df.head())

    # non leaking helpfulness 
    actual_exp_df['base_nonleak_helpful'] = actual_exp_df.apply(lambda x: x.base_correct and ~x.baseline_base_correct and ~x.base_leaked, axis=1)
    actual_exp_df['plus_nonleak_helpful'] = actual_exp_df.apply(lambda x: x.plus_correct and ~x.baseline_plus_correct and ~x.plus_leaked, axis=1)

    row = {"exp_id": exp_id, "model": model}
    for subset in ['base', 'plus']:
        helpful = round(len(actual_exp_df[actual_exp_df[f'{subset}_helpful'] == 1]) / len(actual_exp_df), 3)
        nonleak_helpful = round(len(actual_exp_df[actual_exp_df[f'{subset}_nonleak_helpful'] == 1]) / len(actual_exp_df), 3)
        leak_freq = round(len(actual_exp_df[actual_exp_df[f'{subset}_leaked'] == 1]) / len(actual_exp_df), 3)
        print(f"{subset} helpful: {helpful}, leakage: {leak_freq}, nonleak helpful: {nonleak_helpful}")
        row[f"{subset}_helpfulness"] = helpful
        row[f"{subset}_nonleak_helpfulness"] = nonleak_helpful
        row[f"{subset}_leakage_freq"] = leak_freq
    
   
    all_rows.append(row)

df = pd.DataFrame(all_rows)
df.to_csv(output_file, index=False)
df.head()

Teacher Feedback (actual = mbpp_with_feedback_gpt-3.5-turbo, leak exp = mbpp_just_feedback_gpt-3.5-turbo):
base helpful: 0.185, leakage: 0.627, nonleak helpful: 0.0
plus helpful: 0.143, leakage: 0.526, nonleak helpful: 0.0
Teacher Feedback (No Cheat No Question) (actual = mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question):
base helpful: 0.05, leakage: 0.384, nonleak helpful: 0.0
plus helpful: 0.042, leakage: 0.384, nonleak helpful: 0.0
Teacher Feedback with RAG (actual = mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question):
base helpful: 0.056, leakage: 0.376, nonleak helpful: 0.0
plus helpful: 0.053, leakage: 0.37, nonleak helpful: 0.0
RAG Feedback (actual = mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_retrieval_as_feedback):
base helpful: 0.032, leakage: 0.413, nonleak helpful: 0.0
plus

,exp_id,model,base_helpfulness,base_nonleak_helpfulness,base_leakage_freq,plus_helpfulness,plus_nonleak_helpfulness,plus_leakage_freq
0,Teacher Feedback,allenai--OLMo-2-1124-7B-Instruct,0.185,0.0,0.627,0.143,0.0,0.526
1,Teacher Feedback (No Cheat No Question),allenai--OLMo-2-1124-7B-Instruct,0.050,0.0,0.384,0.042,0.0,0.384
2,Teacher Feedback with RAG,allenai--OLMo-2-1124-7B-Instruct,0.056,0.0,0.376,0.053,0.0,0.370
3,RAG Feedback,allenai--OLMo-2-1124-7B-Instruct,0.032,0.0,0.413,0.034,0.0,0.405
